## Multi-Input Multi-Output Pipeline for Fashion Attributes (Two Datasets)

This notebook does the following:
 - Loads a CSV file with columns for Image_Path, barcode, brand, colors, category, style, gender, pattern, occasion, fit, Type, and lenghth.
 - Splits the data into training and test sets.
 - Determines the number of classes for each multi-label attribute.
 - Prepares tf.data.Datasets for training and testing.
 - Builds a model with three input branches (image, barcode, brand) and nine output heads (all with sigmoid activation).
 - Defines custom F1 loss and metric functions.
 - Trains the model with validation.
 - Evaluates and saves the model.
 - Runs an inference example.
 


In [82]:
import tensorflow as tf
import pandas as pd
import numpy as np
import ast
from sklearn.model_selection import train_test_split

In [84]:
# Read and decode image; resize to 224x224 and normalize
def load_and_preprocess_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)  # adjust if not JPEG
    image = tf.image.resize(image, [224, 224])
    image = image / 255.0
    return image

In [87]:
# Convert a string (e.g. "[0, 1, 0, 1]") into a numpy array of float32
def parse_multi_label(label_str):
    label_str = label_str.numpy().decode('utf-8')
    label_list = ast.literal_eval(label_str)
    return np.array(label_list, dtype=np.float32)

In [88]:
# Process each row of the CSV: load image, cast barcode, and parse each multi-label attribute.
def process_row(image_path, barcode, brand, colors, category, style, gender, pattern, occasion, fit, type_val, length_val):

    image = load_and_preprocess_image(image_path)
    barcode = tf.cast(barcode, tf.float32)
    brand = tf.convert_to_tensor(brand, dtype=tf.string)
    
    colors = tf.py_function(func=parse_multi_label, inp=[colors], Tout=tf.float32)
    colors.set_shape([process_row.num_colors])
    
    category = tf.py_function(func=parse_multi_label, inp=[category], Tout=tf.float32)
    category.set_shape([process_row.num_category])
    
    style = tf.py_function(func=parse_multi_label, inp=[style], Tout=tf.float32)
    style.set_shape([process_row.num_style])
    
    gender = tf.py_function(func=parse_multi_label, inp=[gender], Tout=tf.float32)
    gender.set_shape([process_row.num_gender])
    
    pattern = tf.py_function(func=parse_multi_label, inp=[pattern], Tout=tf.float32)
    pattern.set_shape([process_row.num_pattern])
    
    occasion = tf.py_function(func=parse_multi_label, inp=[occasion], Tout=tf.float32)
    occasion.set_shape([process_row.num_occasion])
    
    fit = tf.py_function(func=parse_multi_label, inp=[fit], Tout=tf.float32)
    fit.set_shape([process_row.num_fit])
    
    type_val = tf.py_function(func=parse_multi_label, inp=[type_val], Tout=tf.float32)
    type_val.set_shape([process_row.num_type])
    
    length_val = tf.py_function(func=parse_multi_label, inp=[length_val], Tout=tf.float32)
    length_val.set_shape([process_row.num_length])
    
    outputs = {
        'color_output': colors,
        'category_output': category,
        'style_output': style,
        'gender_output': gender,
        'pattern_output': pattern,
        'occasion_output': occasion,
        'fit_output': fit,
        'type_output': type_val,
        'length_output': length_val
    }
    return (image, barcode, brand), outputs

In [89]:
# Create a tf.data.Dataset from the DataFrame subset.
def create_dataset(df_subset):
    dataset = tf.data.Dataset.from_tensor_slices((
        df_subset['Image_Path'].values,
        df_subset['barcode'].values,
        df_subset['brand'].values,
        df_subset['colors'].values,
        df_subset['category'].values,
        df_subset['style'].values,
        df_subset['gender'].values,
        df_subset['pattern'].values,
        df_subset['occasion'].values,
        df_subset['fit'].values,
        df_subset['Type'].values,
        df_subset['lenghth'].values
    ))
    dataset = dataset.map(process_row, num_parallel_calls=tf.data.AUTOTUNE)
    return dataset

In [90]:
def build_model(num_colors, num_category, num_style, num_gender, num_pattern, num_occasion, num_fit, num_type, num_length, brand_vocab):
    # Build the multi-input multi-output model.
    from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten, Dropout, concatenate, Embedding
    from tensorflow.keras.models import Model
    
    # Image branch
    image_input = Input(shape=(224, 224, 3), name='image_input')
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(image_input)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Flatten()(x)
    x = Dense(64, activation='relu')(x)
    
    # Barcode branch
    barcode_input = Input(shape=(1,), name='barcode_input')
    b = Dense(16, activation='relu')(barcode_input)
    b = Dense(16, activation='relu')(b)
    
    # Brand branch
    brand_input = Input(shape=(1,), name='brand_input', dtype=tf.string)
    from tensorflow.keras.layers import StringLookup
    brand_lookup = StringLookup(vocabulary=brand_vocab, mask_token=None, num_oov_indices=0)
    brand_index = brand_lookup(brand_input)
    embedding_dim = 8
    brand_emb = Embedding(input_dim=len(brand_vocab) + 1, output_dim=embedding_dim)(brand_index)
    brand_emb = Flatten()(brand_emb)
    brand_emb = Dense(16, activation='relu')(brand_emb)
    
    # Combine branches
    combined = concatenate([x, b, brand_emb])
    combined = Dense(128, activation='relu')(combined)
    combined = Dropout(0.5)(combined)
    
    # Output heads
    color_output    = Dense(num_colors, activation='sigmoid', name='color_output')(combined)
    category_output = Dense(num_category, activation='sigmoid', name='category_output')(combined)
    style_output    = Dense(num_style, activation='sigmoid', name='style_output')(combined)
    gender_output   = Dense(num_gender, activation='sigmoid', name='gender_output')(combined)
    pattern_output  = Dense(num_pattern, activation='sigmoid', name='pattern_output')(combined)
    occasion_output = Dense(num_occasion, activation='sigmoid', name='occasion_output')(combined)
    fit_output      = Dense(num_fit, activation='sigmoid', name='fit_output')(combined)
    type_output     = Dense(num_type, activation='sigmoid', name='type_output')(combined)
    length_output   = Dense(num_length, activation='sigmoid', name='length_output')(combined)
    
    model = Model(
        inputs=[image_input, barcode_input, brand_input],
        outputs=[color_output, category_output, style_output, gender_output,
                 pattern_output, occasion_output, fit_output, type_output, length_output]
    )
    return model

In [92]:
# Custom differentiable F1 loss (1 - F1 score)
def f1_loss(y_true, y_pred):
    epsilon = 1e-7
    y_pred = tf.clip_by_value(y_pred, epsilon, 1 - epsilon)
    tp = tf.reduce_sum(y_true * y_pred, axis=0)
    fp = tf.reduce_sum((1 - y_true) * y_pred, axis=0)
    fn = tf.reduce_sum(y_true * (1 - y_pred), axis=0)
    f1 = 2 * tp / (2 * tp + fp + fn + epsilon)
    return 1 - tf.reduce_mean(f1)

In [91]:
# Custom F1 metric that thresholds predictions at 0.5
def f1_metric(y_true, y_pred):
    epsilon = 1e-7
    y_pred_thresh = tf.cast(tf.greater(y_pred, 0.5), tf.float32)
    tp = tf.reduce_sum(y_true * y_pred_thresh, axis=0)
    fp = tf.reduce_sum((1 - y_true) * y_pred_thresh, axis=0)
    fn = tf.reduce_sum(y_true * (1 - y_pred_thresh), axis=0)
    f1 = 2 * tp / (2 * tp + fp + fn + epsilon)
    return tf.reduce_mean(f1)

In [93]:
def run_pipeline(csv_path):
    print("\nProcessing dataset:", csv_path)
    # Load CSV and split the data
    df_local = pd.read_csv(csv_path)
    train_df_local, test_df_local = train_test_split(df_local, test_size=0.2, random_state=42)
    
    # Determine the number of classes using the entire dataset
    def get_classes(column):
        sample = ast.literal_eval(df_local[column].iloc[0])
        return len(sample)
    
    num_cols = {
        'colors': get_classes('colors'),
        'category': get_classes('category'),
        'style': get_classes('style'),
        'gender': get_classes('gender'),
        'pattern': get_classes('pattern'),
        'occasion': get_classes('occasion'),
        'fit': get_classes('fit'),
        'Type': get_classes('Type'),
        'lenghth': get_classes('lenghth')
    }
    
    # Set these as attributes on process_row so that set_shape can use them
    process_row.num_colors = num_cols['colors']
    process_row.num_category = num_cols['category']
    process_row.num_style = num_cols['style']
    process_row.num_gender = num_cols['gender']
    process_row.num_pattern = num_cols['pattern']
    process_row.num_occasion = num_cols['occasion']
    process_row.num_fit = num_cols['fit']
    process_row.num_type = num_cols['Type']
    process_row.num_length = num_cols['lenghth']
    
    print("Classes per attribute:", num_cols)
    
    # Create training and test datasets
    train_ds = create_dataset(train_df_local)
    train_ds = train_ds.shuffle(buffer_size=len(train_df_local)).batch(16).prefetch(tf.data.AUTOTUNE)
    test_ds = create_dataset(test_df_local).batch(16).prefetch(tf.data.AUTOTUNE)
    
    # Build the model. (Use the brand vocabulary from this dataset.)
    brand_vocab_local = sorted(df_local['brand'].unique())
    model = build_model(num_cols['colors'], num_cols['category'], num_cols['style'],
                        num_cols['gender'], num_cols['pattern'], num_cols['occasion'],
                        num_cols['fit'], num_cols['Type'], num_cols['lenghth'], brand_vocab_local)
    
    # Compile the model with custom F1 loss and metric for each output
    output_names_local = ['color_output', 'category_output', 'style_output', 'gender_output',
                          'pattern_output', 'occasion_output', 'fit_output', 'type_output', 'length_output']
    custom_losses = {name: f1_loss for name in output_names_local}
    custom_metrics = {name: f1_metric for name in output_names_local}
    model.compile(optimizer='adam', loss=custom_losses, metrics=custom_metrics)
    model.summary()
    
    # Train the model with validation
    EPOCHS = 5
    history = model.fit(train_ds, epochs=EPOCHS, validation_data=test_ds)
    
    # Evaluate on test dataset
    eval_results = model.evaluate(test_ds)
    print("\nEvaluation results:", eval_results)
    
    # Save the model
    model_save_path = csv_path.replace(".csv", "_model.h5")
    model.save(model_save_path)
    print("Model saved to:", model_save_path)
    
    # Run inference on one sample (first row of training data)
    sample_row = train_df_local.iloc[0]
    sample_image_path = sample_row['Image_Path']
    sample_barcode = sample_row['barcode']
    sample_brand = sample_row['brand']
    print("\nRunning inference on sample:")
    print("Image:", sample_image_path)
    print("Barcode:", sample_barcode)
    print("Brand:", sample_brand)
    predictions = model.predict([tf.expand_dims(load_and_preprocess_image(sample_image_path), 0),
                                 tf.expand_dims(tf.cast(sample_barcode, tf.float32), 0),
                                 tf.expand_dims(tf.convert_to_tensor(sample_brand, dtype=tf.string), 0)])
    print("Predictions:")
    for name, pred in zip(output_names_local, predictions):
        print(f"{name}: {pred}")
    
    return model, history, eval_results

In [94]:
dataset_paths = "Data/dataset_drop_encoded.csv"
model_obj, hist, eval_res = run_pipeline(dataset_paths)


Processing dataset: Data/dataset_drop_encoded.csv
Classes per attribute: {'colors': 29, 'category': 19, 'style': 6, 'gender': 3, 'pattern': 55, 'occasion': 19, 'fit': 6, 'Type': 36, 'lenghth': 11}


Model: "functional_14"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ image_input         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_41 (Conv2D)  │ (None, 224, 224,  │        896 │ image_input[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_41    │ (None, 112, 112,  │          0 │ conv2d_41[0][0]   │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_42 (Conv2D)  │ (None, 112, 112,  │     18,496 │ max_pooling2d_41… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_42    │ (None, 56, 56,    │          0 │ conv2d_42[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ brand_input         │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_43 (Conv2D)  │ (None, 56, 56,    │     73,856 │ max_pooling2d_42… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ string_lookup_11    │ (None, 1)         │          0 │ brand_input[0][0] │
│ (StringLookup)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_43    │ (None, 28, 28,    │          0 │ conv2d_43[0][0]   │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ barcode_input       │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_15        │ (None, 1, 8)      │        576 │ string_lookup_11… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_29          │ (None, 100352)    │          0 │ max_pooling2d_43… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_76 (Dense)    │ (None, 16)        │         32 │ barcode_input[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_30          │ (None, 8)         │          0 │ embedding_15[0][… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_75 (Dense)    │ (None, 64)        │  6,422,592 │ flatten_29[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_77 (Dense)    │ (None, 16)        │        272 │ dense_76[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_78 (Dense)    │ (None, 16)        │        144 │ flatten_30[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_13      │ (None, 96)        │          0 │ dense_75[0][0],   │
│ (Concatenate)       │                   │            │ dense_77[0][0], 

 Total params: 6,553,016 (25.00 MB)

 Trainable params: 6,553,016 (25.00 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 56s 2s/step - category_output_f1_metric: 0.0771 - category_output_loss: 0.9229 - color_output_f1_metric: 0.0669 - color_output_loss: 0.9331 - fit_output_f1_metric: 0.1918 - fit_output_loss: 0.8082 - gender_output_f1_metric: 0.2336 - gender_output_loss: 0.7664 - length_output_f1_metric: 0.1680 - length_output_loss: 0.8320 - loss: 7.8915 - occasion_output_f1_metric: 0.0972 - occasion_output_loss: 0.9028 - pattern_output_f1_metric: 0.0327 - pattern_output_loss: 0.9673 - style_output_f1_metric: 0.1655 - style_output_loss: 0.8345 - type_output_f1_metric: 0.0755 - type_output_loss: 0.9245 - val_category_output_f1_metric: 0.0320 - val_category_output_loss: 0.9680 - val_color_output_f1_metric: 0.0290 - val_color_output_loss: 0.9710 - val_fit_output_f1_metric: 0.1317 - val_fit_output_loss: 0.8683 - val_gender_output_f1_metric: 0.0695 - val_gender_output_loss: 0.9305 - val_length_output_f1_metric: 0.1165 - val_length_output_loss: 0.8835 - val_loss: 8.2937 - v


Evaluation results: [8.293743133544922, 0.9709956049919128, 0.9679692983627319, 0.8209938406944275, 0.9305468201637268, 0.9857580661773682, 0.8954576253890991, 0.868283748626709, 0.9643455743789673, 0.8835471868515015, 0.032030683010816574, 0.029004313051700592, 0.13171617686748505, 0.06945304572582245, 0.11645282804965973, 0.1045423150062561, 0.01424194686114788, 0.1790061891078949, 0.03565436974167824]
Model saved to: Data/dataset_drop_encoded_model.h5

Running inference on sample:
Image: data/images_droped\image_49015801020101.jpg
Barcode: 49015801020101
Brand: CKS
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 469ms/step
Predictions:
color_output: [[0. 0. 0. 1. 0. 1. 1. 0. 1. 0. 0. 0. 0. 1. 1. 0. 1. 1. 1. 0. 0. 1. 0. 1.
  0. 1. 0. 1. 0.]]
category_output: [[1. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 1. 0. 1. 1. 0. 0.]]
style_output: [[0. 1. 1. 0. 1. 0.]]
gender_output: [[0. 1. 0.]]
pattern_output: [[1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 0. 1. 0. 1. 1. 0. 1. 0.
  0. 1. 0. 0. 0. 0. 1. 0. 1. 1. 1

In [96]:
dataset_paths_2 = "Data/dataset_imputed_encoded.csv"
model_obj_2, hist_2, eval_res_2 = run_pipeline(dataset_paths_2)


Processing dataset: Data/dataset_imputed_encoded.csv
Classes per attribute: {'colors': 29, 'category': 25, 'style': 8, 'gender': 3, 'pattern': 63, 'occasion': 20, 'fit': 7, 'Type': 46, 'lenghth': 17}


Model: "functional_15"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ image_input         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_44 (Conv2D)  │ (None, 224, 224,  │        896 │ image_input[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_44    │ (None, 112, 112,  │          0 │ conv2d_44[0][0]   │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_45 (Conv2D)  │ (None, 112, 112,  │     18,496 │ max_pooling2d_44… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_45    │ (None, 56, 56,    │          0 │ conv2d_45[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ brand_input         │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_46 (Conv2D)  │ (None, 56, 56,    │     73,856 │ max_pooling2d_45… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ string_lookup_12    │ (None, 1)         │          0 │ brand_input[0][0] │
│ (StringLookup)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_46    │ (None, 28, 28,    │          0 │ conv2d_46[0][0]   │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ barcode_input       │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_16        │ (None, 1, 8)      │        680 │ string_lookup_12… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_31          │ (None, 100352)    │          0 │ max_pooling2d_46… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_81 (Dense)    │ (None, 16)        │         32 │ barcode_input[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_32          │ (None, 8)         │          0 │ embedding_16[0][… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_80 (Dense)    │ (None, 64)        │  6,422,592 │ flatten_31[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_82 (Dense)    │ (None, 16)        │        272 │ dense_81[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_83 (Dense)    │ (None, 16)        │        144 │ flatten_32[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_14      │ (None, 96)        │          0 │ dense_80[0][0],   │
│ (Concatenate)       │                   │            │ dense_82[0][0], 

 Total params: 6,557,506 (25.01 MB)

 Trainable params: 6,557,506 (25.01 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
24/24 ━━━━━━━━━━━━━━━━━━━━ 160s 3s/step - category_output_f1_metric: 0.0561 - category_output_loss: 0.9439 - color_output_f1_metric: 0.0591 - color_output_loss: 0.9409 - fit_output_f1_metric: 0.1789 - fit_output_loss: 0.8211 - gender_output_f1_metric: 0.2390 - gender_output_loss: 0.7610 - length_output_f1_metric: 0.1180 - length_output_loss: 0.8820 - loss: 8.1193 - occasion_output_f1_metric: 0.1014 - occasion_output_loss: 0.8986 - pattern_output_f1_metric: 0.0305 - pattern_output_loss: 0.9695 - style_output_f1_metric: 0.0423 - style_output_loss: 0.9577 - type_output_f1_metric: 0.0555 - type_output_loss: 0.9445 - val_category_output_f1_metric: 0.0279 - val_category_output_loss: 0.9721 - val_color_output_f1_metric: 0.0370 - val_color_output_loss: 0.9630 - val_fit_output_f1_metric: 0.2119 - val_fit_output_loss: 0.7881 - val_gender_output_f1_metric: 0.1132 - val_gender_output_loss: 0.8868 - val_length_output_f1_metric: 0.0952 - val_length_output_loss: 0.9048 - val_loss: 8.4015 - 


Evaluation results: [8.401461601257324, 0.963007926940918, 0.9720994830131531, 0.9999999403953552, 0.8867713809013367, 0.9799200892448425, 0.9315283894538879, 0.7880995273590088, 0.9752821326255798, 0.9047525525093079, 0.027900470420718193, 0.03699203208088875, 0.2119004875421524, 0.11322859674692154, 0.09524732828140259, 0.0684715062379837, 0.020079897716641426, 0.0, 0.02471783757209778]
Model saved to: Data/dataset_imputed_encoded_model.h5

Running inference on sample:
Image: data/images_imputed\image_49019601030101.jpg
Barcode: 49019601030101
Brand: CKS
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Predictions:
color_output: [[1. 1. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 1. 0. 1. 0. 1. 0.
  1. 0. 0. 0. 1.]]
category_output: [[1. 0. 0. 0. 0. 0. 1. 1. 0. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 0. 1. 0. 0. 0.
  1.]]
style_output: [[0. 0. 0. 0. 1. 0. 0. 0.]]
gender_output: [[0. 1. 1.]]
pattern_output: [[0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 1. 1. 1. 0. 1. 0. 1. 0. 1. 1. 0.
  1. 1. 1. 0. 1. 0. 0.

In [102]:
import os
import json
import pandas as pd

class MetadataSaver:
    def __init__(self, save_path="metadata"):
        self.save_path = save_path
        if not os.path.exists(self.save_path):
            os.makedirs(self.save_path)

    def save_metadata(self, item_id, metadata, file_format="json"):
        if file_format == "json":
            file_path = os.path.join(self.save_path, f"{item_id}.json")
            with open(file_path, "w") as file:
                json.dump(metadata, file, indent=4)
            print(f"Metadata saved in {file_path}")
        
        elif file_format == "csv":
            file_path = os.path.join(self.save_path, f"{item_id}.csv")
            df = pd.DataFrame([metadata])
            df.to_csv(file_path, index=False)
            print(f"Metadata saved in {file_path}")
        
        else:
            print("Invalid file format! Please use 'json' or 'csv'.")

In [101]:
# Example usage after training
save_model_metadata(model_obj, history, eval_results, dataset_path="Data/dataset_droped.csv", output_path="metadata_dataset_droped.json")

AttributeError: 'InputLayer' object has no attribute 'get_output_shape_at'